### Monte Carlo Option pricing using Python libraries


Due to the complicated nature of the barrier and price algorithmic averaging, there is no analytical solution for this example of [exotic option](https://www.investopedia.com/terms/e/exoticoption.asp). We can use the Monte Carlo simulation method to estimate the expected value of profit on the maturity day. Traditionally, Monte Carlo pricing is done in the C/C++ CUDA code. In this notebook, we will show this can be done efficiently in the Python libraries like Numba and CuPy.

Following are the parameters we choose to price the example Asian Barrier Option:

    Maturity (T): 1 year
    Spot (S) : 120
    Strike (K): 110
    Volatility (sigma): 35.0 %
    Risk Free Rate (r): 5.0 %
    Stock Drift Rate (mu): 10.0 %
    Barrier (B): 100
    
To run this notebook successfully, you need to use V100 GPUs.  

### CUDA Monte Carlo Option Pricing

Traditionally, the Monte Caro Option pricing is implemented in CUDA C/C++. Following is one example,

In [1]:
from IPython.display import Markdown as md
f = open('cuda_pricing.cu', 'r')
md("""```C
%s
   ```""" % (f.read()))

```C
#include <vector>
#include <stdio.h>
#include <iostream>
#include <chrono>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <curand.h>
 
#define CHECKCURAND(expression)                         \
  {                                                     \
    curandStatus_t status = (expression);                         \
    if (status != CURAND_STATUS_SUCCESS) {                        \
      std::cerr << "Curand Error on line " << __LINE__<< std::endl;     \
      std::exit(EXIT_FAILURE);                                          \
    }                                                                   \
  }

// atomicAdd is introduced for compute capability >=6.0
#if !defined(__CUDA_ARCH__) || __CUDA_ARCH__ >= 600
#else
__device__ double atomicAdd(double* address, double val)
{
      printf("device arch <=600\n");
        unsigned long long int* address_as_ull = (unsigned long long int*)address;
          unsigned long long int old = *address_as_ull, assumed;
            do {
                    assumed = old;
                        old = atomicCAS(address_as_ull, assumed,
                                                    __double_as_longlong(val + __longlong_as_double(assumed)));
                          } while (assumed != old);
              return __longlong_as_double(old);
}
#endif

__global__ void sumPayoffKernel(float *d_s, const unsigned N_PATHS, double *mysum)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;

  extern __shared__ double smdata[];
  smdata[tid] = 0.0;

  for (unsigned i = idx; i<N_PATHS; i+=stride)
  {
    smdata[tid] += (double) d_s[i];
  }

  for (unsigned s=blockDim.x/2; s>0; s>>=1)
  {
    __syncthreads();
    if (tid < s) smdata[tid] += smdata[tid + s];
  }

  if (tid == 0)
  {
    atomicAdd(mysum, smdata[0]);
  }
}

__global__ void barrier_option(
    float *d_s,
    const float T,
    const float K,
    const float B,
    const float S0,
    const float sigma,
    const float mu,
    const float r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  const float tmp1 = mu*T/N_STEPS;
  const float tmp2 = exp(-r*T);
  const float tmp3 = sqrt(T/N_STEPS);
  double running_average = 0.0;

  for (unsigned i = idx; i<N_PATHS; i+=stride)
  {
    float s_curr = S0;
    for(unsigned n = 0; n < N_STEPS; n++){
       s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS];
       running_average += (s_curr - running_average) / (n + 1.0) ;
       if (running_average <= B){
           break;
       }
    }

    float payoff = (running_average>K ? running_average-K : 0.f);
    d_s[i] = tmp2 * payoff;
  }
}

int main(int argc, char *argv[]) {
  try {
    // declare variables and constants
    size_t N_PATHS = 8192000;
    size_t N_STEPS = 365;
    if (argc >= 2)  N_PATHS = atoi(argv[1]);

    if (argc >= 3)  N_STEPS = atoi(argv[2]);

    const float T = 1.0f;
    const float K = 110.0f;
    const float B = 100.0f;
    const float S0 = 120.0f;
    const float sigma = 0.35f;
    const float mu = 0.1f;
    const float r = 0.05f;


    double gpu_sum{0.0};

    int devID{0};
    cudaDeviceProp deviceProps;

    checkCudaErrors(cudaGetDeviceProperties(&deviceProps, devID));
    printf("CUDA device [%s]\n", deviceProps.name);
    printf("GPU Device %d: \"%s\" with compute capability %d.%d\n\n", devID, deviceProps.name, deviceProps.major, deviceProps.minor);
    // Generate random numbers on the device
    curandGenerator_t curandGenerator;
    CHECKCURAND(curandCreateGenerator(&curandGenerator, CURAND_RNG_PSEUDO_MTGP32));
    CHECKCURAND(curandSetPseudoRandomGeneratorSeed(curandGenerator, 1234ULL)) ;

    const size_t N_NORMALS = (size_t)N_STEPS * N_PATHS;
    float *d_normals;
    checkCudaErrors(cudaMalloc(&d_normals, N_NORMALS * sizeof(float)));
    CHECKCURAND(curandGenerateNormal(curandGenerator, d_normals, N_NORMALS, 0.0f, 1.0f));
    cudaDeviceSynchronize();

  	// before kernel launch, check the max potential blockSize
  	int BLOCK_SIZE, GRID_SIZE;
  	checkCudaErrors(cudaOccupancyMaxPotentialBlockSize(&GRID_SIZE,
  	                                                   &BLOCK_SIZE,
  	                                                   barrier_option,
  	                                                   0, N_PATHS));

  	std::cout << "suggested block size " << BLOCK_SIZE
  	          << " \nsuggested grid size " << GRID_SIZE
  	          << std::endl;

  	std::cout << "Used grid size " << GRID_SIZE << std::endl;

  	// Kernel launch
  	auto t1=std::chrono::high_resolution_clock::now();

  	float *d_s;
  	checkCudaErrors(cudaMalloc(&d_s, N_PATHS*sizeof(float)));

  	auto t3=std::chrono::high_resolution_clock::now();
  	barrier_option<<<GRID_SIZE, BLOCK_SIZE>>>(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS);
  	cudaDeviceSynchronize();
  	auto t4=std::chrono::high_resolution_clock::now();

  	double* mySum;
  	checkCudaErrors(cudaMallocManaged(&mySum, sizeof(double)));
  	sumPayoffKernel<<<GRID_SIZE, BLOCK_SIZE, BLOCK_SIZE*sizeof(double)>>>(d_s, N_PATHS, mySum);
  	cudaDeviceSynchronize();
  	auto t5=std::chrono::high_resolution_clock::now();

  	std::cout << "sumPayoffKernel takes "
  	          << std::chrono::duration_cast<std::chrono::microseconds>(t5-t4).count() / 1000.f
  	          << " ms\n";

  	gpu_sum = mySum[0] / N_PATHS;

  	auto t2=std::chrono::high_resolution_clock::now();

  	// clean up
  	CHECKCURAND(curandDestroyGenerator( curandGenerator )) ;
  	checkCudaErrors(cudaFree(d_s));
  	checkCudaErrors(cudaFree(d_normals));
  	checkCudaErrors(cudaFree(mySum));

  	std::cout << "price "
              << gpu_sum
              << " time "
  	          << std::chrono::duration_cast<std::chrono::microseconds>(t5-t1).count() / 1000.f
  	          << " ms\n";
  }

  catch(std::
        exception& e)
  {
    std::cout<< "exception: " << e.what() << "\n";
  }
}

   ```

The CUDA code is usually long and detailed. In general, it is performing a sequence of 5 tasks:
1. Allocate GPU memory to store the random number and simulation path results
2. Call cuRand library to generate random numbers
3. Launch the barrier option kernel to do parallel simulations
4. Launch the sum kernel to aggregate the terminal derivative prices.
5. Deallocate the memory

Developers have to perform each step explicitly. 

Compile and run the code:

In [2]:
!make out
!./out

make: 'out' is up to date.
CUDA device [Tesla V100-SXM2-16GB]
GPU Device 0: "Tesla V100-SXM2-16GB" with compute capability 7.0

suggested block size 1024 
suggested grid size 160
Used grid size 160
sumPayoffKernel takes 1.275 ms
price 18.7026 time 23.029 ms


Compiling and running this CUDA code on a V100 GPU produces the correct option price $18.70 in 22.05ms for 8.192 million paths and 365 steps. We will use these numbers as our reference benchmark for later comparison. Among the 5 steps, the critical component is step 3, where data scientists need to describe the detailed Monte Carlo simulation. Ideally the data scientists efforts should be focused on this step. 

## Python Monte Carlo Option Pricing
We set the constants for the option and load the necessary libraries:-

In [3]:
import cupy
import numpy as np
import math
import time
import numba
from numba import cuda
from numba import njit
from numba import prange
import cudf

N_PATHS = 8192000
N_STEPS = 365
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05

As we know the [Standard Error of the Mean](https://en.wikipedia.org/wiki/Standard_error) is proportional to the inversed square root of the number of samples. Hence the more simulation paths we have, the more accurate the pricing will be. We will simulate 8.192 million paths with 365 steps where each step represents a day. 

#### Single Thread CPU
The single thread CPU code for the Monte Carlo simulation has two nested for-loops. The outer loop iterates each path while the inner loop iterates time and computes the underlying asset price for that day. Note that this code is accelerated via [Numba @jit](http://numba.pydata.org/) hence it compiles into machine code at runtime. 

In [4]:
@njit(fastmath=True)
def cpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)
    running_average = 0.0
    for i in range(N_PATHS):
        s_curr = S0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break

        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

  We use CuPy to generate Gaussian random numbers in the GPU and allocate an array to store the prices at maturity.


In [5]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output =  np.zeros(N_PATHS, dtype=np.float32)

Now we will run the Monte Carlo simulation and time it. When the Numba accelerated function is called for the first time, there is some overhead to compile it. So to time it accurately, we run this method twice and and consider the run time of the second attempt. 


In [6]:
cpu_barrier_option(output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
s = time.time()
cpu_barrier_option(output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
v = output.mean()
e = time.time()
print('time', e-s, 'v', v)

time 27.66077470779419 v 18.706627


#### Multiple Cores CPU
CPU has multiple cores and to make a fair comparison, the code can be modified a little to take advantage of all the CPU cores. Note how we parallelize the outer loop:-

In [7]:
@njit(fastmath=True, parallel=True)
def cpu_multiplecore_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)
    for i in prange(N_PATHS):
        s_curr = S0
        running_average = 0.0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break
        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

Running this parallel code and timing it:-

In [8]:
cpu_multiplecore_barrier_option(output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
s = time.time()
cpu_multiplecore_barrier_option(output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
v = output.mean()
e = time.time()
print('time', e-s, 'v', v)

time 1.3220970630645752 v 18.706627


We see aproximately 32x speedup due to 32 cores of the CPU. 

#### NUMBA GPU
The multiple cores CPU code can be modified easily to run in the GPU via Numba.cuda.jit. The code below is very similar to the CPU multiple core code except that we parallize the outer loop on the GPU. Running this code and timing it:-

In [9]:
@cuda.jit
def numba_gpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)
    running_average = 0.0
    for i in range(ii, N_PATHS, stride):
        s_curr = S0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average += (s_curr - running_average) / (n + 1.0)
            if running_average <= B:
                break
        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [10]:

number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
s = time.time()
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v)

time 0.06206059455871582 v 18.706621


We get 4x speedup compared to the mutliple core version and 128x speedup compared to the single core version. 

#### NUMBA Shared Memory 
While accessing the global memory for Gaussian random numbers, the memory access is already aligned and numbers are only read once. So using shared memory is not helping the performace as shown below:-

In [11]:
@cuda.jit
def numba_gpu_barrier_option_shared_mem(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # load to shared memory
    path_offset = cuda.blockIdx.x * cuda.blockDim.x
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)
    running_average = 0.0
    for i in range(ii, N_PATHS, stride):
        s_curr = S0
        for n in range(N_STEPS):
            shared[cuda.threadIdx.x] = d_normals[path_offset + cuda.threadIdx.x + n * N_PATHS]
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*shared[cuda.threadIdx.x]
            running_average += (s_curr - running_average) / (n + 1.0)
            if running_average <= B:
                break
        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [12]:
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
shared_buffer_size = number_of_threads * 4
numba_gpu_barrier_option_shared_mem[(number_of_blocks,), (number_of_threads,), 0, shared_buffer_size](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
s = time.time()
numba_gpu_barrier_option_shared_mem[(number_of_blocks,), (number_of_threads,), 0, shared_buffer_size](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v)

time 0.06257963180541992 v 18.706621


#### CUPY GPU
CuPy provides an easy way to define GPU kernels from raw CUDA source. `RawKernel` object allows you to call the kernel with CUDA’s `cuLaunchKernel` interface. Here is an example where we wrap the Barrier Option computation code inside the `RawKernel`:

In [13]:
cupy_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void barrier_option(
    float *d_s,
    const float T,
    const float K,
    const float B,
    const float S0,
    const float sigma,
    const float mu,
    const float r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;

  const float tmp1 = mu*T/N_STEPS;
  const float tmp2 = exp(-r*T);
  const float tmp3 = sqrt(T/N_STEPS);
  double running_average = 0.0;

  for (unsigned i = idx; i<N_PATHS; i+=stride)
  {
    float s_curr = S0;
    unsigned n=0;
    for(unsigned n = 0; n < N_STEPS; n++){
       s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS];
       running_average += (s_curr - running_average) / (n + 1.0) ;
       if (running_average <= B){
           break;
       }
    }

    float payoff = (running_average>K ? running_average-K : 0.f);
    d_s[i] = tmp2 * payoff;
  }
}

''', 'barrier_option')

We can launch it to compute the same Barrier Option price:-

In [14]:
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
s = time.time()
cupy_barrier_option((number_of_blocks,), (number_of_threads,),
                   (output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r),  randoms_gpu, N_STEPS, N_PATHS))
v = output.mean()
cupy.cuda.stream.get_current_stream().synchronize()
e = time.time()
print('time', e-s, 'v',v)

time 0.025598764419555664 v 18.706621


This approach is the most efficient way to use the GPU and it achieves 8x speedup compared to the 32 core CPU performance. Compared with CUDA C/C++ approach (31ms), CuPy performance (35ms) is very close to it.

### Multiple GPUs Option Pricing

To get a more accurate estimation of the option price, more paths are needed for Monte Carlo simulation. The single V100 GPU we used in the above example only has 32GB memory and we are hitting the memory limits to run 8M simulations. [DASK](https://dask.org/) is an integrated component of RAPIDS for distributed computation on GPUs.  We can take advantage of it to distribute the Monte Carlo simulation computation to multiple nodes across multiple GPUs. First, we need to wrap all the computation inside a function to allow the allocated GPU memory to be released at the end of the function call. Note that the function takes an extra argument for the random number seed value so the individual function calls each have an independent sequence of random numbers. Loading the DASK library and setting up the local CUDA cluster :-

In [15]:
# clear the GPU memory
del randoms_gpu 
del randoms_cpu
del output


def get_option_price(T, K, B, S0, sigma, mu, r, N_PATHS = 8192000, N_STEPS = 365, seed=3):
    number_of_threads = 256
    number_of_blocks = (N_PATHS-1) // number_of_threads + 1
    cupy.random.seed(seed)
    randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
    output =  cupy.zeros(N_PATHS, dtype=cupy.float32)
    cupy_barrier_option((number_of_blocks,), (number_of_threads,),
                   (output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r),  randoms_gpu, N_STEPS, N_PATHS))
    v = output.mean()
    out_df = cudf.DataFrame()
    out_df['p'] = cudf.Series([v.item()])
    return out_df
o = get_option_price(T=1.0, K=120.0, B=90.0, S0=100.0, sigma=0.2, mu=0.1, r=0.05)

In [16]:
import dask
import dask_cudf
from dask.delayed import delayed
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46441 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 540.94 GB


There are 4 GPUs inside the system. To distribute the above function, we wrap it into the `delayed` function to integrate it into the DASK computation graph. We use `from_delayed` to gather all the distributed dataframes into a holistic cudf_dask dataframe. We can call the cudf_dask dataframe `mean` and `std` to calculate the expected mean and standard deviation of the prices.

In [17]:
# for 32G card
NUM_PATHS=8192000
# for 16G card
NUM_PATHS=2048000

x = dask_cudf.from_delayed([delayed(get_option_price)(T=1.0, K=110.0, B=100.0, S0=120.0, sigma=0.35, mu=0.1, r=0.05, N_PATHS=NUM_PATHS, seed=3000+i) for i in range(1600)])

In [18]:
x.mean().compute()

p    18.710948
dtype: float64

In [19]:
x.std().compute()

p    0.014738
dtype: float64

The code computed 1600 Monte Carlo simulations of `8192000` paths. By averaging the price together to get a better estimation, the standard deviation is reduced by a factor of 1/sqrt(1600) = 1/40  